# Mishra Data Quality Index 3 (p3.py)
Max Calzada 
January 8, 2022 (Latest Update)

'dqi4_4sentences.csv' [snli_1.0_test.jsonl](https://github.com/uclnlp/inferbeddings/blob/master/data/snli/snli_1.0_test.jsonl.gz) and must be manually inserted into the content folder each session.

## What does DQI c3 do?

DQI c3 measures Inter-Sample STS (Semantic Textual Similarity). In other words, DQI c3 measures how similar the different sentences/data are to each other. 

The aim is for the sentences to have a low amount of variation in similarity to each other.

## User Instructions

DQI c3 may require some manual edits by the user, depending on the user's needs. The following steps guide the user in editing the code if necessary.

### STEP 1: Hyperparameter

Under the `@jit def func2():` the line of code reads `a=1`. In this case, `a=1` means that the user wants one sentence/datum to yield the minimum similarity score. 

If the user were to edit the code to read `a=3`, this would mean that the user wants 3 sentence/data to yield the minimum similarity score.

### STEP 2: tf2

If tf2 yields `None`, this means that the value of DQI c3 is only `tf1` and that `tf2` does not factor in. In this case leave code as is and ignore STEP 3.

If tf2 yields a numerical value, proceed to STEP 3.

### STEP 3: dqic3

If tf2 yields a numerical value, comment in the lines `dqic3=tf1+tf2` and `print(dqic3)`.

In this case, the value of DQI c3 will be yielded by `dqic3`.

In [ ]:
import pandas as pd # Load libraries
import numpy as np 
import json
import csv

from tqdm import tqdm 
from tqdm import tqdm_notebook as tqdm 

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('averaged_perceptron_tagger')

import spacy
import string
import sys
import random
import pickle

from collections import Counter
from itertools import chain

nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

import spacy.cli

spacy.cli.download("en_core_web_lg") # https://www.google.com/search?q=what+does+en_core_web_lg+do&sxsrf=ALiCzsYQ__E0ulCClCOKwhb8uwQYY1bcRw%3A1668697353810&ei=CU12Y5byMIKs5NoP-8KCyA4&ved=0ahUKEwjWmouIvrX7AhUCFlkFHXuhAOkQ4dUDCBA&uact=5&oq=what+does+en_core_web_lg+do&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQAzIECAAQRzIECAAQRzIECAAQRzIECAAQRzIECAAQRzIECAAQRzIECAAQRzIECAAQRzoKCAAQRxDWBBCwA0oECEEYAEoECEYYAFD_A1j_A2DjDGgBcAJ4AIABAIgBAJIBAJgBAKABAcgBCMABAQ&sclient=gws-wiz-serp
nlp = spacy.load("en_core_web_lg") # tokenize, lemmatize, and summarize

from sklearn.metrics.pairwise import cosine_similarity 

from spacy.lang.en.examples import sentences  

from numba import jit, cuda 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
# This code was borrowed from DQI c1.

df_wino = pd.read_csv('dqi4_4sentences.csv')
words2=pd.DataFrame()
frames=[df_wino['documents']]
words2['full'] = pd.concat(frames)
words2.reset_index(inplace = True) 
words2['casewords'] = words2['full'].str.lower()
words2['wsw'] = words2['casewords'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
words2['nopunc'] = words2['wsw'].str.replace("'", "")
words2['nopunc'] = words2['nopunc'].str.replace(".", "")
words2['nopunc'] = words2['nopunc'].str.replace(",", "")
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text): # finds roots of words in text
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)] 
words2['text_lemmatized'] = words2.nopunc.apply(lemmatize_text) # ISSUE : lemmatize_text # package updates?

words2['final']="NaN"
def final(x):
    makeitastring = ' '.join(map(str, x))
    makeitastring=makeitastring.replace(",","")
    makeitastring=makeitastring.replace("'","")
    makeitastring=makeitastring.replace("[","")
    makeitastring=makeitastring.replace("]","")
    return makeitastring
for i in tqdm(range(len(words2))):    
    column=['text_lemmatized']
    row=[i]
    words2.loc[i,'final']=final(words2.loc[i,'text_lemmatized'])
words2['fullnopunc'] = words2['full'].str.replace("'", "")
words2['fullnopunc'] = words2['fullnopunc'].str.replace(".", "")
words2['fullnopunc'] = words2['fullnopunc'].str.replace(",", "")
words2['label']='NaN'

<ipython-input-2-9c3dcdc17983>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  words2['nopunc'] = words2['nopunc'].str.replace(".", "")
<ipython-input-2-9c3dcdc17983>:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(words2))):


  0%|          | 0/4 [00:00<?, ?it/s]

<ipython-input-2-9c3dcdc17983>:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  words2['fullnopunc'] = words2['fullnopunc'].str.replace(".", "")


In [ ]:
words=pd.read_csv('dqi4_4sentences.csv')
snlitrain_l_list = [] # empty list
pandas_json_attempt = []
with open('./snli_1.0_test.jsonl') as snlitrain_file_pointer: # snli_1.0_test.jsonl
    for item in snlitrain_file_pointer: # open `snli_1.0_test.jsonl`
        snlitrain_l_list.append(item) # `append()` places `snli_1.0_test.jsonl` items in list

In [ ]:
data = []
for item in snlitrain_l_list:
    data.append(json.loads(item)) # parse a valid JSON string and convert it to a Python Dictionary
      # and appends this to `data`
df_snlitrain = pd.DataFrame.from_dict(data) # constructs DataFrame from dict of array-like or dicts
nlp = spacy.load("en_core_web_lg") # spacy.load("en_trf_bertbaseuncased_lg")

@jit
def func():
    x=[]
    for l in range(len(data)-1):
        f=0
        token_l=nlp(data[l]) # tokenizes sentence `words`?
        for m in range(len(data)-1): 
            if(m!=l): # different sentence
                token_m=nlp(data[m]) # tokenizes another sentences
                slm=token_l.similarity(token_m) # measures similarity be                   
                diff=SIML-slm # subracts similarity of two sentences from threshold.
                if(diff<=0):
                    f=f+1
        x.append(f)
    t = pd.Series(x)
    tf1=t.var()
    return tf1

@jit
def amaxelements(list1, N): # creates variables `list1` and `N`.
    sa=0
    for i in range(0, N):
        max1 = 0  
        for j in range(len(list1)):
            if list1[j] > max1: 
                max1 = list1[j]; 
        list1.remove(max1); 
        sa=sa+max1
    return sa

# STEP 1: Set hyperparameter `a=` to many sentences should have the minimum simialrity score
@jit
def func2():
    a=1 # Hyperparameter
    sl=0
    for l in range(len(data)-1):
        x=[] # empty set
        token_l=nlp(data[l]) # tokenizes sentences
        for m in range(len(data)-1): 
            if(m!=l): # different sentence
                token_m=nlp(data[m]) 
                slm=token_l.similarity(token_m) # similarity                
                diff1=SIML-slm 
                diff2=abs(diff1)
                diff=abs(diff1-diff2)
                x.append(diff) # x creates vector/list of differences
        sl=sl+amaxelements(x,a)
    tf2=sl/5

In [ ]:
words2['final']

0    solubility measurement correlation îµ-2 4 6 8 ...
1    combustion behavior flame structure high energ...
2    effect particle size thermal decomposition clâ...
3    ï»¿energetic characteristic calculation new ge...
Name: final, dtype: object

In [ ]:
data=words2['final']
SIML=0.8

tf1 = func()
print(tf1) 

# STEP 2: If tf2 yields "None", leave code as is and ignore STEP 3.
#   If tf2 yields a numerical value, proceed to STEP 3.
tf2 = func2()
print(tf2)

# # STEP 3: Comment in the following two lines.
# dqic3=tf1+tf2
# print(dqic3)

<ipython-input-23-11bb4ef93c50>:8: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "func" failed type inference due to: Untyped global name 'data': Cannot determine Numba type of <class 'pandas.core.series.Series'>

File "<ipython-input-23-11bb4ef93c50>", line 11:
def func():
    <source elided>
    x=[]
    for l in range(len(data)-1): # tqdm(range(len(data)-1)): #
    ^

  @jit
<ipython-input-23-11bb4ef93c50>:8: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "func" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-23-11bb4ef93c50>", line 11:
def func():
    <source elided>
    x=[]
    for l in range(len(data)-1): # tqdm(range(len(data)-1)): #
    ^

  @jit
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "func" was compiled in object mode without force

0.0


<ipython-input-23-11bb4ef93c50>:39: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "func2" failed type inference due to: Untyped global name 'data': Cannot determine Numba type of <class 'pandas.core.series.Series'>

File "<ipython-input-23-11bb4ef93c50>", line 43:
def func2():
    <source elided>
    sl=0
    for l in range(len(data)-1): # tqdm(range(len(data)-1)): # 
    ^

  @jit
<ipython-input-23-11bb4ef93c50>:39: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "func2" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-23-11bb4ef93c50>", line 43:
def func2():
    <source elided>
    sl=0
    for l in range(len(data)-1): # tqdm(range(len(data)-1)): # 
    ^

  @jit
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "func2" was compiled in object mode with

None


## How should the DQI c3 results be interpreted?

`dqi4_4sentences.csv` yielded values of `0` and `None`. In a sense, `0 + None = 0`.

In his paper, Mishra classified values at 14.3370 and below as "good" and values at and above 16.7024 as "bad", meaning that the value of 0 is classified as "good".

In the case of `dqi4_4sentences.csv`, this means that its sentences/data are all very distinct from each other.